In [39]:
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.tree import DecisionTreeClassifier

In [40]:
path = '/home/xuan/Downloads/archive/credit_card_transactions.csv'

In [41]:
df = pd.read_csv(path)

In [42]:
#Explore data

In [43]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,28705.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,NaN
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,83236.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,NaN
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,22844.0


In [44]:
df.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.100702e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03,4.682575e+04
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02,2.583400e+04
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00,1.001000e+03
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00,2.511400e+04
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00,4.586000e+04
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00,6.831900e+04
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00,9.940300e+04


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [46]:
df.shape

(1296675, 24)

In [47]:
print("Number of unique values in each column: ")
for col in df.columns:
    print(f"{col}: {df[col].nunique()}")

Number of unique values in each column: 
Unnamed: 0: 1296675
trans_date_trans_time: 1274791
cc_num: 983
merchant: 693
category: 14
amt: 52928
first: 352
last: 481
gender: 2
street: 983
city: 894
state: 51
zip: 970
lat: 968
long: 969
city_pop: 879
job: 494
dob: 968
trans_num: 1296675
unix_time: 1274823
merch_lat: 1247805
merch_long: 1275745
is_fraud: 2
merch_zipcode: 28336


In [48]:
null_counts = df.isnull().sum()
null_columns = null_counts[null_counts > 0]
if len(null_columns) > 0:
    for col, count in null_columns.items():
        print(f"{col}: {count}")
else:
    print("There are no null values in the DataFrame.")

merch_zipcode: 195973


In [49]:
print("Number of duplicate rows:", df.duplicated().sum())

Number of duplicate rows: 0


In [50]:
df = df.drop(columns=['trans_date_trans_time', 'trans_num', 'unix_time', 'dob', 'first', 'last'])

# Handle missing values
df['merch_zipcode'] = df['merch_zipcode'].fillna(-1)

In [51]:
train, test = train_test_split(df, test_size=0.2)

In [52]:
file = test.drop(["is_fraud"], axis = 1)
file.to_csv("/home/xuan/vscode/kafka/test.csv")

In [53]:
X = train.drop(['is_fraud'], axis=1)
y = train['is_fraud']

In [54]:
value_counts = y.value_counts()
print(value_counts)

is_fraud
0    1031380
1       5960
Name: count, dtype: int64


In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 42)

In [56]:
categorical_features = ['merchant', 'category', 'gender', 'city', 'state', 'job']

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), ['amt', 'cc_num', 'zip', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long', 'merch_zipcode'])  # Adjust numerical columns as needed
    ]
)

In [57]:
parameter = { 
    "classifier__max_depth": [None, 2, 5, 10],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__min_samples_leaf": [1, 2, 5],
    "classifier__max_features": [None, "sqrt", "log2"]
}

In [58]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(criterion = 'entropy',
                                          max_depth = 10, 
                                          min_samples_split = 10, 
                                          min_samples_leaf = 5,
                                          max_features = None))])

In [59]:
# gscv = GridSearchCV(pipeline, param_grid = parameter, cv = 5, n_jobs = -1, verbose = 2)
# gscv.fit(X_train, y_train)

In [60]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['merchant', 'category',
                                                   'gender', 'city', 'state',
                                                   'job']),
                                                 ('num', StandardScaler(),
                                                  ['amt', 'cc_num', 'zip',
                                                   'lat', 'long', 'city_pop',
                                                   'merch_lat', 'merch_long',
                                                   'merch_zipcode'])])),
                ('classifier',
                 DecisionTreeClassifier(criterion='entropy', max_depth=10,
                                        min_samples_leaf=5,
                                        min_samples_split=10))])

In [61]:
y_pred = pipeline.predict(X_test)

In [62]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))

Accuracy: 0.9969778471860721
F1_score:  0.7263203841117416
Confusion Matrix:
 [[206009    229]
 [   398    832]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    206238
           1       0.78      0.68      0.73      1230

    accuracy                           1.00    207468
   macro avg       0.89      0.84      0.86    207468
weighted avg       1.00      1.00      1.00    207468



In [63]:
with open("/home/xuan/vscode/kafka/model.pkl", "wb") as file:
    pickle.dump(pipeline, file)